In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
import numpy as np
import pandas as pd
from pathlib import Path
import argparse
from sklearn.model_selection import GridSearchCV
import yaml
import shutil
import datetime

In [2]:
print("Num of GPUs available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num of GPUs available:  0


In [3]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [4]:
tf.config.run_functions_eagerly(True)

In [5]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
    shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

In [6]:
X_train.shape[1]

25

In [7]:
BATCH_SIZE = [32, 64, 128]
BUFFER_SIZE = 256
LEARNING_RATE = [0.05, 0.1, 0.0025]
EPOCHS = 200
NUMBER_OF_NEURONS = [256, 128, 256]

In [8]:
class NotNotMyModel(Model):
    def __init__(self, n_of_neurons=64):
        super(NotNotMyModel, self).__init__()

        self.d_in = Dense(25, activation='relu')
        self.d_1 = Dense(n_of_neurons, activation = 'relu')
        self.d_2 = Dense(n_of_neurons, activation = 'relu')
        self.d_3 = Dense(n_of_neurons, activation = 'relu')
        self.d_out = Dense(1)

    def call(self, x):
        x = self.d_in(x)
        x = self.d_1(x)
        x = self.d_2(x)
        x = self.d_3(x)
        x = self.d_out(x)
        return x

In [9]:

top_mae = 100
top_params = []
loss_object = tf.keras.losses.MeanSquaredError()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')


In [10]:
def get_log_dirs(params):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = logs_path / params / 'gradient_tape' / current_time / 'train'
    train_log_dir.mkdir(exist_ok=True, parents=True)
    test_log_dir = logs_path / params / 'gradient_tape' / current_time / 'test'
    test_log_dir.mkdir(exist_ok=True, parents=True)
    fit_log_dir = logs_path / params / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    fit_log_dir.mkdir(exist_ok=True, parents=True)

    return train_log_dir, test_log_dir, fit_log_dir

In [11]:
@tf.function
def train_step(input_vector, labels, model, optimizer, loss_object):
    with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(input_vector, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [12]:
@tf.function
def test_step(input_vector, labels, model, loss_object):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [13]:
for batch in BATCH_SIZE:
    #print(batch)
    train_ds = tf.data.Dataset.from_tensor_slices(
        (pd.read_csv(X_train_name), pd.read_csv(y_train_name))
    ).shuffle(BUFFER_SIZE).batch(batch)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (pd.read_csv(X_test_name), pd.read_csv(y_test_name))
    ).batch(batch)
    for lr in LEARNING_RATE:
        #optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        for n_neurons in NUMBER_OF_NEURONS:
            params = 'ba{}_lr{}_nn{}'.format(batch, lr, n_neurons)
            train_log_dir, test_log_dir, logdir= get_log_dirs(params)
            train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
            test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))
            fit_summary_writer = tf.summary.create_file_writer(str(logdir))
            print("Params are: BATCH_SIZE={}, LEARNING_RATE={}, NUMBER_OF_NEURONS={}".format(batch, lr, n_neurons))
            tf.summary.trace_on(graph=True, profiler=True)
            model = NotNotMyModel(n_of_neurons=n_neurons)
            optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
            #print(getattr(model, 'x'))

            for epoch in range(EPOCHS):
                for (x_train, y_train) in train_ds:
                    with fit_summary_writer.as_default():
                        train_step(x_train, y_train, model, optimizer, loss_object)

                with train_summary_writer.as_default():
                    tf.summary.scalar('loss', train_loss.result(), step=epoch)
                    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

                for (x_test, y_test) in test_ds:
                    test_step(x_test, y_test, model, loss_object)

                with test_summary_writer.as_default():
                    tf.summary.scalar('loss', test_loss.result(), step=epoch)
                    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

                template = 'Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
                print (template.format(epoch+1,
                                       train_loss.result(),
                                       train_accuracy.result(),
                                       test_loss.result(),
                                       test_accuracy.result()))
                if epoch+1 == EPOCHS:
                    if top_mae > test_accuracy.result():
                        top_params = [batch, lr, n_neurons]
                        top_mae = test_accuracy.result()
                        model.save('data/models/MyNN', overwrite=True)

                train_loss.reset_states()
                train_accuracy.reset_states()
                test_loss.reset_states()
                test_accuracy.reset_states()

            with fit_summary_writer.as_default():
                tf.summary.trace_export(
                    name = "my_func_trace",
                    step=0,
                    profiler_outdir=logdir
                )



Params are: BATCH_SIZE=32, LEARNING_RATE=0.05, NUMBER_OF_NEURONS=256
Instructions for updating:
use `tf.profiler.experimental.start` instead.


2022-12-19 20:47:12.167840: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2022-12-19 20:47:12.167847: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


Epoch: 1, Loss: 190052384.0, Accuracy: 1531.12353515625, Test Loss: 19963.69140625, Test MAE: 68.697021484375
Epoch: 2, Loss: 12137.6767578125, Accuracy: 51.14064407348633, Test Loss: 1942.1239013671875, Test MAE: 38.69796371459961
Epoch: 3, Loss: 1477.829833984375, Accuracy: 29.976032257080078, Test Loss: 646.085205078125, Test MAE: 21.04400062561035
Epoch: 4, Loss: 318.42828369140625, Accuracy: 12.879393577575684, Test Loss: 194.6702880859375, Test MAE: 9.992147445678711
Epoch: 5, Loss: 238.3948211669922, Accuracy: 10.898737907409668, Test Loss: 303.43511962890625, Test MAE: 12.44105052947998
Epoch: 6, Loss: 258.8653564453125, Accuracy: 11.194354057312012, Test Loss: 325.6803894042969, Test MAE: 12.312715530395508
Epoch: 7, Loss: 220.8506622314453, Accuracy: 10.475004196166992, Test Loss: 164.28378295898438, Test MAE: 8.238277435302734
Epoch: 8, Loss: 174.3322296142578, Accuracy: 9.5321044921875, Test Loss: 94.2801284790039, Test MAE: 7.611474990844727
Epoch: 9, Loss: 180.42803955078

KeyboardInterrupt: 

In [ ]:
print(top_params)